In [2]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Đọc tệp dữ liệu Cancer.csv trong file data_cancer
df = pd.read_csv('/home/vitquay1708/Study_Space/ml/week2/data/food_order.csv')

print(df.head() )


         cuisine     city       device user_segment    promo fav_ingredient  \
0       japanese  city_19          web      student  promo_8         sesame   
1        chinese   city_1          ios       budget  promo_5            soy   
2  mediterranean  city_14  desktop_app       family  promo_8         yogurt   
3        italian  city_15       tablet          new  promo_4          lemon   
4     vietnamese  city_10          ios          new  promo_8          basil   

  spice_level day_of_week price_bucket  
0      medium         Tue          mid  
1   extra_hot         Wed    very_high  
2        none         Mon          mid  
3        none         Sat          mid  
4        mild         Tue          mid  


###### Tính các xác suất p_k := N_k / N (chú ý N = len(df) )

In [3]:
target_feature = 'cuisine'

Y = df[target_feature] 
labels = np.zeros((len(set(Y)), 2), dtype=Y.dtype) 

# Liệt kê các nhãn (label) khác nhau (từ trường cuisine) k =1, 2, ... C
# và tính xác suất P_k của mỗi nhãn. Sử dụng nhãn thay cho chỉ số
id = 0
for label in set(Y):
    labels[id, 0] = label
    labels[id, 1] = (Y == label).sum()/len(Y)
    id += 1

###### Lấy tên mỗi trường (field) của dữ liệu và số lựa chọn ứng với trường đó

In [24]:

num_labels_M = np.array([['cuisine', 10],
                         ['city', 20],
                         ['device', 5],
                         ['user_segment', 6],
                         ['promo', 9],
                         ['fav_ingredient', 18],
                         ['spice_level', 5],
                         ['day_of_week', 7],
                         ['price_bucket', 4]] )

print(num_labels_M)

[['cuisine' '10']
 ['city' '20']
 ['device' '5']
 ['user_segment' '6']
 ['promo' '9']
 ['fav_ingredient' '18']
 ['spice_level' '5']
 ['day_of_week' '7']
 ['price_bucket' '4']]


###### Tính các xác suất thành phần P(xi | k) theo công thức xấp xỉ (qua Smooth Laplacian) P(xi| y = k) ~ lambda_ik := [|{xm in class k : x_im = xi}| + alpha] / [N_k + M*alpha]

In [ ]:
def Pxik_feature_per_class(Xk, xi, alpha = 1.0):
# tính toán cho toàn bộ mảng thay vì dùng vòng lặp
    X_ = np.array(Xk.iloc[:, 1:])
    xi =  np.array(xi)
    Pxik = np.zeros(len(xi))
    M_nomials = (num_labels_M[1:, 1]).astype(int)
    N_k = len(Xk)
    count = (X_ == xi).sum()
    Pxik = (count + alpha)/(N_k + M_nomials*alpha)
    
#Returns the prob. of feature per class
    return np.array(Pxik, dtype=float)
    

###### Tính argmax_k { ln[P(k)*P(x|k)]} =  argmax_k { ln[P(k)]  + ln[P(x_1|k)] + ln[P(x_2|k)] + ... + ln[P(x_d|k)]}

In [6]:
def predict_output_label(X, x_input, target_feature):
    #score for each class Pk
    p = np.log(np.array(labels[:, 1], dtype=float))

    for k in range(len(p)):

        Xk = X[X[target_feature] == labels[k, 0]]

        P_xik = Pxik_feature_per_class(Xk, x_input)
        p[k] += np.sum(np.log(P_xik))
        
    y_star = np.argmax(p)
    return labels[y_star, 0]

In [7]:
Train_size = round(len(df)*0.7)
X_Train = df[:Train_size]
Y_Train = df[target_feature][:Train_size]

X_Test = df[Train_size:].drop(target_feature, axis = 1).reset_index(drop=True, inplace=False)
Y_Test = df[target_feature][Train_size:].reset_index(drop=True, inplace=False)

Y_pred = np.zeros(len(Y_Test), dtype = Y_Test.dtype) 
print(len(X_Test), len(Y_Test))

for i in range(len(X_Test)):
    Y_pred[i] = predict_output_label(X_Train, X_Test.iloc[i, :], target_feature)
    if(i % 50 == 0):
        print(i, Y_pred[i], Y_Test[i])

900 900
0 chinese chinese
50 mexican japanese
100 chinese chinese
150 thai thai
200 american american
250 thai italian
300 chinese japanese
350 indian indian
400 american american
450 korean japanese
500 mediterranean italian
550 thai indian
600 mediterranean mexican
650 chinese japanese
700 american american
750 mediterranean italian
800 thai vietnamese
850 vietnamese american


In [8]:
from sklearn.metrics import accuracy_score

print(accuracy_score(Y_pred, Y_Test))

0.4122222222222222


In [9]:
print(predict_output_label(X_Train, X_Train.iloc[3, 1:], target_feature) )

italian


In [21]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.naive_bayes import MultinomialNB

X_Train_n = X_Train.drop(target_feature, axis = 1).reset_index(drop=True, inplace=False)

le = OneHotEncoder()
for col in X_Train_n.columns:
    print(X_Train_n[col])
    # X_Train_n[col] = le.fit_transform(X_Train_n[col])


Y_Train = df[target_feature][:Train_size]

X_Test = df[Train_size:].drop(target_feature, axis = 1).reset_index(drop=True, inplace=False)
Y_Test = df[target_feature][Train_size:].reset_index(drop=True, inplace=False)

# for col in X_Test.select_dtypes(include=['object', 'category']).columns:
#     X_Test[col] = le.fit_transform(X_Test[col])

# clf = MultinomialNB()

# clf.fit(X_Train_n, Y_Train)
# y_pred = clf.predict(X_Test)

# print(accuracy_score(y_pred, Y_Test))

0       city_19
1        city_1
2       city_14
3       city_15
4       city_10
         ...   
2095     city_7
2096    city_19
2097    city_10
2098     city_9
2099     city_3
Name: city, Length: 2100, dtype: object
0               web
1               ios
2       desktop_app
3            tablet
4               ios
           ...     
2095            web
2096    desktop_app
2097         tablet
2098            web
2099        android
Name: device, Length: 2100, dtype: object
0         student
1          budget
2          family
3             new
4             new
          ...    
2095          vip
2096    returning
2097    returning
2098    returning
2099          new
Name: user_segment, Length: 2100, dtype: object
0       promo_8
1       promo_5
2       promo_8
3       promo_4
4       promo_8
         ...   
2095    promo_7
2096    promo_5
2097    promo_5
2098    promo_2
2099       none
Name: promo, Length: 2100, dtype: object
0        sesame
1           soy
2        yogurt
3         l

In [22]:
print(X_Train_n)

         city       device user_segment    promo fav_ingredient spice_level  \
0     city_19          web      student  promo_8         sesame      medium   
1      city_1          ios       budget  promo_5            soy   extra_hot   
2     city_14  desktop_app       family  promo_8         yogurt        none   
3     city_15       tablet          new  promo_4          lemon        none   
4     city_10          ios          new  promo_8          basil        mild   
...       ...          ...          ...      ...            ...         ...   
2095   city_7          web          vip  promo_7          lemon        none   
2096  city_19  desktop_app    returning  promo_5          basil        none   
2097  city_10       tablet    returning  promo_5        oregano        mild   
2098   city_9          web    returning  promo_2         tomato         hot   
2099   city_3      android          new     none         ginger        none   

     day_of_week price_bucket  
0            Tue   